TODO (in this version):
- Clean Description text
- Enable user input of Strain

In [135]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import FeatureHasher
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

Data Reading and Cleaning

In [136]:
path = r"C:\\Users\\Luke Melto\\Documents\\GitHub\\Med-Cabinet-BW\\data\\cannabis.csv"
data = pd.read_csv(path)

In [137]:
data.head()

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."


In [138]:
data.describe()

,Rating
count,2351.000000
mean,4.310634
std,0.836394
min,0.000000
25%,4.200000
50%,4.400000
75%,4.700000
max,5.000000


In [139]:
#TODO clean description text

data.columns = [i.strip() for i in data.columns.to_list()]
data.Strain = [i.strip() for i in data.Strain]
data.Flavor = [i.strip() for i in data.Flavor]
data.Effects = [i.strip() for i in data.Effects]

In [140]:
print(data.Flavor.isnull().values.sum())
data["Flavor"] = data["Flavor"].fillna("None")

0


In [141]:
total_flavor = []
max_len = 0
total_flavor.append("None")
for val in data["Flavor"].values:
    print(val)
    val = val.split(",")
    
    if len(val) > max_len:
        max_len = len(val)
    
    for f in val:
        if f not in total_flavor:
            total_flavor.append(f)

rry
Pine,Citrus,Earthy
Sweet,Earthy,Woody
Earthy,Flowery,Grape
Honey,Earthy,Woody
Apple,Minty,Berry
Mango,Earthy,Sweet
Mango,Tropical,Earthy
Pepper,Spicy/Herbal,Sweet
Mango,Sweet,Tropical
Mango,Sweet,Citrus
Mango,Tropical,Citrus
Sweet,Tropical,Pineapple
Spicy/Herbal,Earthy,Cheese
Sweet,Butter,Woody
Sweet,Berry,Earthy
Minty,Sweet,Berry
Sweet,Grapefruit,Honey
Berry,Earthy,Sweet
Sweet
Pine,Woody,Citrus
Sweet,Woody,Earthy
Sweet,Citrus,Grapefruit
None
Earthy,Pungent,Skunk
Skunk,Spicy/Herbal,Diesel
Earthy,Pungent,Woody
Earthy,Pine,Pungent
Skunk,Lemon,Honey
Pungent,Sweet,Earthy
Earthy,Woody,Pungent
Berry,Sweet,Earthy


Citrus,Sweet,Tropical
Berry,Citrus,Pungent
Sweet,Citrus,Pungent
Citrus,Tropical,Skunk
Pungent,Tropical,Earthy
Mango,Sweet,Diesel
Tropical,Flowery,Menthol
Tropical,Pineapple,Citrus
Earthy,Spicy/Herbal,Pine
Pungent,Earthy,Spicy/Herbal
Flowery,Sweet
Earthy,Sweet,Spicy/Herbal
Sweet,Earthy,Skunk
Berry,Woody,Earthy
Earthy,Citrus,Diesel
Earthy,Blueberry,Pungent
Berry,Strawberry,Grape


In [142]:
print(len(total_flavor))
print(max_len)

51
4


In [143]:
print(data["Effects"].isnull().values.sum())

0


In [144]:
total_effects = []
max_len = 0
for val in data["Effects"].values:
    print(val)
    val = val.split(",")
    
    if len(val) > max_len:
        max_len = len(val)
        
    for e in val:
        if e not in total_effects:
            total_effects.append(e)

laxed,Euphoric,Uplifted,Sleepy
Relaxed,Euphoric,Sleepy,Happy,Tingly
Relaxed,Creative,Giggly,Talkative,Uplifted
Euphoric,Hungry,Relaxed
Euphoric,Relaxed,Uplifted,Happy,Hungry
Happy,Uplifted,Energetic,Relaxed,Creative
None
Euphoric,Talkative,Uplifted,Creative,Energetic
Relaxed,Happy,Uplifted,Giggly,Euphoric
Relaxed,Happy,Hungry,Creative,Euphoric
Relaxed,Euphoric,Sleepy,Happy,Uplifted
Uplifted,Relaxed,Energetic,Euphoric,Focused
Relaxed,Sleepy,Hungry,Aroused,Uplifted
Euphoric,Happy,Relaxed,Energetic,Talkative
Uplifted,Happy,Aroused,Tingly,Creative
Uplifted,Focused,Happy,Energetic,Euphoric
Euphoric,Relaxed,Uplifted,Happy,Energetic
Relaxed,Tingly,Happy,Focused,Hungry
Uplifted,Energetic,Euphoric,Relaxed,Aroused
Relaxed,Euphoric,Focused,Happy,Tingly
Relaxed,Happy,Euphoric,Focused,Uplifted
Relaxed,Happy,Sleepy,Euphoric,Tingly
Talkative,Uplifted,Relaxed,Euphoric,Hungry
Happy,Euphoric,Relaxed,Creative,Energetic
Happy,Relaxed,Uplifted,Euphoric,Talkative
Relaxed,Uplifted,Focused,Energetic,Euphoric


In [145]:
print(len(total_effects))
print(max_len)

16
5


Split Flavor Column

In [146]:
split_data = data["Flavor"].str.split(",")
split_data = split_data.to_list()
names = ["Flavor_1", "Flavor_2", "Flavor_3", "Flavor_4"]
new_flavor = pd.DataFrame(split_data, columns=names)

In [147]:
new_flavor = new_flavor.fillna("None")

In [148]:
new_flavor["Flavor_4"].isnull().values.sum()

# 2308 values of Flavor_4 column are Null, therefore dropping it.
new_flavor = new_flavor.drop(columns=["Flavor_4"])

new_flavor

,Flavor_1,Flavor_2,Flavor_3
0,Earthy,Sweet,Citrus
1,Flowery,Violet,Diesel
2,Spicy/Herbal,Sage,Woody
3,Apricot,Citrus,Grapefruit
4,Citrus,Earthy,Orange
...,...,...,...
2346,Earthy,Woody,Pine
2347,Sweet,Berry,Grape
2348,Earthy,Sweet,Spicy/Herbal
2349,Sweet,Earthy,Pungent


Splitting Effects Column

In [149]:
split_data = data["Effects"].str.split(",").to_list()
names = ["Effect_1", "Effect_2", "Effect_3", "Effect_4", "Effect_5"]

new_effect = pd.DataFrame(split_data, columns=names).fillna("None")

In [150]:
new_effect

,Effect_1,Effect_2,Effect_3,Effect_4,Effect_5
0,Creative,Energetic,Tingly,Euphoric,Relaxed
1,Relaxed,Aroused,Creative,Happy,Energetic
2,Uplifted,Happy,Relaxed,Energetic,Creative
3,Tingly,Creative,Hungry,Relaxed,Uplifted
4,Happy,Relaxed,Euphoric,Uplifted,Talkative
...,...,...,...,...,...
2346,Happy,Uplifted,Relaxed,Euphoric,Energetic
2347,Relaxed,Happy,Euphoric,Uplifted,Sleepy
2348,Relaxed,Sleepy,Talkative,Euphoric,Happy
2349,Relaxed,Sleepy,Euphoric,Happy,Hungry


In [151]:
new_effect.describe()

,Effect_1,Effect_2,Effect_3,Effect_4,Effect_5
count,2351,2351,2351,2351,2351
unique,15,15,14,14,14
top,Relaxed,Happy,Euphoric,Euphoric,Uplifted
freq,825,571,413,386,273


Feature Hasher

In [152]:
effect_hasher = FeatureHasher(n_features=3, input_type="string")
x = effect_hasher.fit_transform(total_effects)

In [153]:
print(x.toarray())
len(np.unique(x.toarray(), axis=0))

[[-3.  3.  0.]
 [-2.  3. -2.]
 [-2.  0.  0.]
 [-1.  3. -2.]
 [ 1.  3.  3.]
 [-1.  6.  0.]
 [-2.  2.  1.]
 [-3.  1.  2.]
 [-1.  4. -1.]
 [-2.  0.  1.]
 [-1.  2. -1.]
 [ 0.  1. -1.]
 [-1.  4. -2.]
 [-2.  3.  1.]
 [ 0.  2.  1.]
 [-1.  3. -1.]]


16

In [154]:
flavor_hasher = FeatureHasher(n_features=10, input_type="string")
y = flavor_hasher.fit_transform(total_flavor)

In [155]:
print(y.toarray())
len(np.unique(y.toarray(), axis=0))

[[ 0.  0.  0.  1. -1.  0.  0. -1.  0.  1.]
 [ 1.  0.  0.  0.  1.  0.  0.  1.  0. -1.]
 [ 0. -1.  0.  0.  0.  0.  0.  0. -1.  1.]
 [ 0. -2.  1.  0.  0.  1.  0.  0.  0. -2.]
 [ 0. -1.  1.  1.  1.  1.  0.  0. -1.  1.]
 [ 0. -1.  1.  1.  0.  0.  0.  0.  0.  1.]
 [ 0. -1.  1.  0.  0.  0.  1.  0.  0.  1.]
 [ 1. -2.  1.  0.  1.  1. -1. -1. -1.  3.]
 [ 1.  0. -1.  0.  0.  0.  0.  0. -1.  1.]
 [ 0.  0.  0.  2.  0.  0.  0.  0.  0.  1.]
 [ 0. -1.  1.  1.  0.  1. -1. -1.  0. -1.]
 [ 1. -1.  1.  1.  0.  2. -1.  0.  0.  1.]
 [ 1.  0.  0.  0. -1.  1.  0.  0.  0.  1.]
 [ 0. -1.  0.  0. -2.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  1. -1.  0.  0.  2.]
 [ 0. -2.  0.  0. -1.  0.  0.  0.  0.  1.]
 [ 0. -2.  1.  0. -1.  0.  0.  0. -1.  0.]
 [ 0.  0. -1.  0.  1.  2.  0.  0.  0.  1.]
 [ 0. -1.  0.  0.  0.  1. -2.  0.  0.  2.]
 [ 1.  0.  1.  1. -1. -1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  1.]
 [ 0. -1.  0.  0.  0. -1.  0.  0.  0.  2.]
 [ 1. -2.  1.  0.  0. -1.  1. -1.  0.  1.]
 [ 2.  0. -

51

Convert Flavor and Effects to numeric data

In [156]:
names = ["Effect_1", "Effect_2", "Effect_3", "Effect_4", "Effect_5"]
num_effect = pd.DataFrame([])
for name in names:
    d = new_effect[name]
    temp = effect_hasher.transform(d).toarray()
    temp = pd.DataFrame(temp)
    num_effect = pd.concat([num_effect, temp], axis=1)

In [157]:
new_effect.head(3)

,Effect_1,Effect_2,Effect_3,Effect_4,Effect_5
0,Creative,Energetic,Tingly,Euphoric,Relaxed
1,Relaxed,Aroused,Creative,Happy,Energetic
2,Uplifted,Happy,Relaxed,Energetic,Creative


In [158]:
names = ["Flavor_1", "Flavor_2", "Flavor_3"]
num_flavor = pd.DataFrame([])
for name in names:
    d = new_flavor[name]
    temp = flavor_hasher.transform(d).toarray()
    temp = pd.DataFrame(temp)
    num_flavor = pd.concat([num_flavor, temp], axis=1)

In [159]:
num_flavor.head(3)

,0,1,2,3,4,5,6,7,8,9,...,0,1,2,3,4,5,6,7,8,9
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,-1.0,...,0.0,-2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,-2.0
1,0.0,-1.0,1.0,1.0,1.0,1.0,0.0,0.0,-1.0,1.0,...,0.0,-1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1.0,-2.0,1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,3.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0


In [160]:
new_data = data.copy()

In [161]:
new_data = pd.get_dummies(new_data, columns=["Type"])
new_data

,Strain,Rating,Effects,Flavor,Description,Type_hybrid,Type_indica,Type_sativa
0,100-Og,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,1,0,0
1,98-White-Widow,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,1,0,0
2,1024,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...,0,0,1
3,13-Dawgs,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...,1,0,0
4,24K-Gold,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%...",1,0,0
...,...,...,...,...,...,...,...,...
2346,Zeus-Og,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...,1,0,0
2347,Zkittlez,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...,0,1,0
2348,Zombie-Kush,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...,0,1,0
2349,Zombie-Og,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...,0,1,0


In [162]:
new_data = new_data.drop(columns=["Strain", "Effects", "Flavor", "Description"])
new_data = pd.concat([new_data, num_effect, num_flavor], axis=1)
new_data

,Rating,Type_hybrid,Type_indica,Type_sativa,0,1,2,0,1,2,...,0,1,2,3,4,5,6,7,8,9
0,4.0,1,0,0,-3.0,3.0,0.0,-2.0,3.0,-2.0,...,0.0,-2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,-2.0
1,4.7,1,0,0,1.0,3.0,3.0,-1.0,6.0,0.0,...,0.0,-1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,4.4,0,0,1,-3.0,1.0,2.0,-2.0,2.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4.2,1,0,0,-2.0,0.0,0.0,-3.0,3.0,0.0,...,1.0,-1.0,1.0,1.0,0.0,2.0,-1.0,0.0,0.0,1.0
4,4.6,1,0,0,-2.0,2.0,1.0,1.0,3.0,3.0,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2346,4.7,1,0,0,-2.0,2.0,1.0,-3.0,1.0,2.0,...,0.0,-2.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0
2347,4.6,0,1,0,1.0,3.0,3.0,-2.0,2.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0,0.0,2.0
2348,5.0,0,1,0,1.0,3.0,3.0,-2.0,3.0,1.0,...,1.0,-2.0,1.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,3.0
2349,4.4,0,1,0,1.0,3.0,3.0,-2.0,3.0,1.0,...,0.0,-1.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0


In [163]:
#TODO - replace this cell with input from user

test_values = [ 4.,  1.,  0.,  0., -3.,  3.,  0., -2.,  3., -2., -2.,  0.,  0., -1.,  3., -2.,  1.,  3.,  3.,  1.,  0.,  0.,  0.,  1.,  0.,  0., 1.,  0., -1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  1., 0., -2.,  1.,  0.,  0.,  1.,  0.,  0.,  0., -2.]

In [164]:
kmeans = KMeans(n_jobs = -1, n_clusters = 30, init='k-means++', max_iter=500)
model = kmeans.fit(new_data)
test_values = np.array(test_values)
test_values = np.reshape(test_values, (1, -1))
model.predict(test_values)

array([0])

In [165]:
nbrs = NearestNeighbors(n_neighbors=5).fit(new_data)

In [166]:
dist, recommends = nbrs.kneighbors(test_values)
print(dist)
print(recommends)

[[0.         4.74236228 4.79583152 5.48087584 5.59016994]]
[[   0  100   24 2205  487]]


In [167]:
for index in recommends[0]:
    print(data.values[index])

['100-Og' 'hybrid ' 4.0 'Creative,Energetic,Tingly,Euphoric,Relaxed'
 'Earthy,Sweet,Citrus'
 '$100 OG is a 50/50 hybrid strain that packs a strong punch. The name supposedly refers to both its strength and high price when it first started showing up in Hollywood. As a plant, $100 OG tends to produce large dark green buds with few stems. Users report a strong body effect of an indica for pain relief with the more alert, cerebral feeling thanks to its sativa side.']
['American-Dream' 'hybrid ' 4.7
 'Focused,Euphoric,Tingly,Energetic,Relaxed' 'Earthy,Sweet,Pungent'
 'American Dream by Sensi Seeds is an indica-dominant strain that blends Afghan Skunk with Jamaican and Hawaiian genetics. Its name is charged with meaning, mainly in that this indica represents “honest toil, personal improvement, and enjoying the fruits of one’s labor,” as Sensi Seeds puts it. Earthy and skunky in flavor, American Dream helps you pursue happiness with balanced uplifting, social effects appropriate for day or e